# 7.1 ODE I

#### Before we start:
* review Assignment 2 model solution

#### Today's class:

* Ordinary differential equations
    - Euler step
    - Discretisation
* Miscellaneous
    - `map`
    - Multi-threaded processing
    - Animation, make movie with ffmpeg


## Ordinary differential equations

Differential equations are often hard to solve on paper but in many cases become trivial on a computer. 

### Euler step
Take the simplest, first order ODE
$$
y^\prime = f(y,x)
$$
where the right-hand side (RHS) is the function $f(y,x)$ that specifies the derivative $y^\prime = \frac{dy}{dx}$. We are looking for the function $y(x)$, but here not the algebraic expression but the numerical values. For a time dependent problem $x = t$. 

Take for example $f (y,x) = 2x$, then we know that $y(x) = x^2$. The differential equation is

$$\frac{dy}{dx} = 2x $$ 

Let's pretend we do not know the answer, but the initial conditions $y(0) = 0$. How can we numerically calculate $y(x)$ for a series of discrete values $x_i$?


### Discretization

We need to turn the ODE into a difference equation:

$$\frac{y_\mathrm{n+1} - y_\mathrm{n}}{x_\mathrm{n+1} - x_\mathrm{n}} = f(y_\mathrm{n},x_\mathrm{n}) $$

which we solve for $y_\mathrm{n+1}$:

$$y_\mathrm{n+1} = y_\mathrm{n} +  h f(y_\mathrm{n},x_\mathrm{n})$$ where $h= x_\mathrm{n+1} - x_\mathrm{n}$

The initial conditions then imply that $X_0 = 0$ and $y_0 = 0$. Specifically then for our case the first few steps look like this:

$$
 \frac{y_1 - y_0}{h} = f(y_0,x_0) ,
$$ solve for $y_1$ and then start stepping:
$$
 y_1  = y_0 + h 2 x_0 \\
 y_2  = y_1 + h 2 x_1 \\
 \dots
$$

This is the **explicit** discretization, or the **Euler** step, the right-hand side is evaluated for the known values $(y_\mathrm{n},x_\mathrm{n})$.

Therefore, we evaluate $f(x,y)$ at a sequence of chosen points `x = [x[0],x[1],x[2],x[3],...,x[n]]` and start according to $ y(0) = 0 $   with `y[0] = 0` and then proceed to 
```python
y[1]  = y[0] + h*2*x[0]
y[2]  = y[1] + h*2*x[1]
y[3]  = y[2] + h*2*x[2]
...
y[n]  = y[n-1] + h*2*x[n-1]
```

Let's implement this:

In [ ]:
%pylab ipympl

In [ ]:
y=[]; y.append(0)
x=[]; x.append(0)

rhs_f = lambda x: 2*x
x_thing = x[0]; y_thing = y[0]
dx=0.75; x_end = 7.
while x_thing <= x_end+dx:
    y_thing = y_thing + dx * rhs_f(x_thing)
    x_thing += dx
    #print(x_thing,y_thing)
    x.append(x_thing); y.append(y_thing)

In [ ]:
close(4);figure(4)
plot(x,y,'o-',label='Euler explicit')
plot(x,array(x)**2,'--',label='analytic')
legend();xlabel('$x$'),ylabel('$y(x)$')

The numerical answer does not agree very well with the analytic answer. By now you are familiar with the question: How can the accuracy be improved?

### Solution using ODE solver library

In [ ]:
from scipy import integrate

In [ ]:
#integrate.odeint?

In [ ]:
rhs_ff = lambda y,x: 2*x
x = linspace(0,8,3)
y0=0

In [ ]:
yy = integrate.odeint(rhs_ff,y0,x)


In [ ]:
figure(4)
plot(x,yy,'v:',label='scipy.integrate.odeint')
legend()

## Miscellaneous

### map
[map](https://docs.python.org/3/library/functions.html#map)
is one of the [built-in Python functions](https://docs.python.org/3/library/functions.html) 
[map](https://www.geeksforgeeks.org/python-map-function/). It returns an iterator that applies function to every item of iterable, yielding the results. `map` in combination with the `lambda` function provides a very similar (if not the same) functionality as list comprehension.

**Example 1:**

In [ ]:
my_pets = ['alfred', 'tabitha', 'william', 'arla']

In [ ]:
my_pets[0].upper()

In [ ]:
[pet.upper() for pet in my_pets]

In [ ]:
list(map(str.upper, my_pets))

**Example 2:**

In [ ]:
f = lambda x,y,z: 100*x + 10*y + z
f(1,2,3)

In [ ]:
x=linspace(1,3,3)
y=linspace(4,6,3)
z=linspace(7,9,3)
print(x,y,z)

In [ ]:
xx = array([1,2,3])
# xx = array([1])
# xx = array([1,2])
f(xx,y,z)

In [ ]:
[f(xx,y,z) for xx in range(1,3)]

In [ ]:
f_ = lambda x: f(xx,y,z)

In [ ]:
[f_(x) for x in range(1,3)]

In [ ]:
cc = map(f_,range(2,4)) 

In [ ]:
cc?

In [ ]:
array(list(cc))

**Example 3:** 

In [ ]:
C = [39.2, 36.5, 37.3, 38, 37.8] 
F = list(map(lambda x: (float(9)/5)*x + 32, C))
print("{:7s}  {:10s}".format('Celsius','Fahrenheit'))
for a,b, in zip(C,F): print("{:7.2f}  {:10.2f}".format(a,b))

### Multiprocessing
In lecture 5.2 we saw that the MC integration, if done for millions of runs, starts to use measurable computing time. This is an example for a case where a particular task needs to be done over and over again, maybe in another situation with different input parameters. Such problems are called _embarassingly parallel_, essentially because it takes no or very little effort to parallelize them. The following introduces a multiprocessing approach that will use multiple cpu cores (see notebook 3.1 where we covered the hardware aspects). 

The following very simple code example demonstrates the use of the [multiprocessing](https://docs.python.org/2/library/multiprocessing.html) module.

Note: Using the `pool` functionality requires that the `__main__` module be importable by the children. This means that `pool` will not work properly in an interactive session. Thus, the following would have to be used as a Python script:

```Python
from multiprocessing import Pool

def f(x):
    return x*x

p = Pool(5)
print(p.map(f, [1, 2, 3]))
```

In [ ]:
def f(x):
    return x*x

print(list(map(f, [1, 2, 3])))

#### Example MC integration

In [ ]:
%pylab ipympl

In [ ]:
def mcint(func,xrange,n):
    '''MC integration of function func over xrange'''
    dx = diff(xrange)[0]
    x = dx*random.rand(n)-xrange[0]
    favg = func(x).mean()
    I = favg * dx
    return I

In [ ]:
func = lambda x: sin(1/(x*(2-x)))**2

In [ ]:
xrange=(0,2); n = 1001   # nmax = 1e6
x = linspace(*xrange,n)
ifig=1; close(ifig);figure(ifig)
plot(x,func(x))

In [ ]:
mcint(func,xrange,n)

In [ ]:
%%time
ints2 = []
nruns = int(5e5)
nmc=1000
for n in range(nruns):
    ints2.append(mcint(func,xrange,nmc))
print("Mean: {:6.4f}  Variance: {:6.4f}".format(mean(ints2),var(ints2)))

`nruns` determines how many times we are going to determine the integral with a complete MC integration. Remember, each time we will get a slightly different answer because each time a different set of random numbers will be drawn. Each MC integration will use `nmc` random numbers. 

As we can see, doing a million MC integrals takes about a minute. However, we are only using one of our 12 cores. We can use some more with the `multiporcessing` package. 

```Python
from multiprocessing import Pool
import numpy as np

nruns = int(5e6)

def mc(i):
    '''MC integration of function func over xrange'''
    n = 1000
    dx = 2.
    x = dx*np.random.rand(n)
    y = np.sin(1/(x*(2-x)))**2
    favg = y.mean()
    I = favg * dx
    return I


p = Pool(20)
ints3 = np.array(p.map(mc,range(nruns)))
print(ints3.mean(),ints3.var())
```

Run the above program and compare the performance:

In [ ]:
# single process
34.5/nruns

In [ ]:
# 4 procs
4* 74 / 5e6 

In [ ]:
# 10 procs
10* 44 / 5e6 

### Animation, make movie with ffmpeg

A really easy way to make an animation is to generate a sequence of numbered `.png` images and connect them with the command-line program ``ffmpeg`. 

#### Example: Oscillating sine 
We want to make a movie of an oscillating spline.

In [ ]:
ifig=2
close(ifig)
figure(ifig)

In [ ]:
x = linspace(0,2*pi)
ff = 0.25
plot(x, sin(ff*2*pi)*sin(x),lw=0.8)
ylabel('$\sin x$'), xlabel('$x$')

In [ ]:
ifig=3;close(ifig);figure(ifig)
x = linspace(0,2*pi)
fact = linspace(0,2.,32)
for ff in fact:
    plot(x, sin(ff*pi)*sin(x),lw=0.8)
    ylabel('$\sin x$'), xlabel('$x$')

In [ ]:
color=['c', 'm', 'y', 'k','r','g','b']
linestyle=['-', '--', ':', '-.']
cl = [a+b for a in linestyle for b in color]

In [ ]:
from random import shuffle
shuffle(cl)

In [ ]:
ifig=4;close(ifig);figure(ifig)
x = linspace(0,2*pi)
fact = linspace(0,2.,32)
for i,ff in enumerate(fact):
    plot(x, sin(ff*pi)*sin(x),cl[mod(i,len(cl))],lw=0.8)
    ylabel('$\sin x$'), xlabel('$x$')

With the following command the plot can be saved into a png image file:

In [ ]:
savefig('my_great_sin_plot.png')

Now, make one plot per line and save the plot each time as a png image file. 

In [ ]:
ifig=4;close(ifig);figure(ifig)
x = linspace(0,2*pi)
fact = linspace(0,2.,40)
for i,ff in enumerate(fact):
    ifig=4;close(ifig);figure(ifig)
    plot(x, sin(ff*pi)*sin(x),'r-',lw=0.8)
    ylim(-1.1,1.1)
    savefig('sin'+str(i).zfill(4)+".png")
    ylabel('$\sin x$'), xlabel('$x$')

In [ ]:
!rm sin*png

In [ ]:
!ls sin*png

We can use the ffmpeg command to generate out of these frames a movie:
```
#!/bin/bash
# Produces mp4 movie from series of images (e.g. png format)
# ARG1: name base of image files that is followed by numbers, and prefix.
#       for example if the images are named img0001.png than ARG1 is "img"

ffmpeg  -framerate 10   -y -f image2  -pattern_type glob -i "$1*.png" -preset slow -crf 18  -c:v libx264 -b:v 12000k  -pix_fmt yuv420p  $1.mp4

```

In [ ]:
!cat ./bin/movie.sh

In [ ]:
%%bash
./bin/movie.sh sin

This great, but the movie does not look good yet. We need more frames for better time resolution. The notebook environment is not suitable for this, and we can only make one plot at a time. So, let's use multiprocessing again. The python script would look like this:

```Python
from multiprocessing import Pool
from matplotlib import pyplot as pl
import numpy as np

x = np.linspace(0,2*np.pi)
fact = np.linspace(0,5,2000)

def make_image(inp):
    i,ff = inp
    pl.plot(x, np.sin(ff*np.pi)*np.sin(x),'r-',lw=0.8)
    pl.ylabel('$\sin x$'), pl.xlabel('$x$')
    pl.ylim(-1.05,1.05)
    pl.savefig('sin'+str(i).zfill(4)+".png")

p = Pool(10)
p.map(make_image,enumerate(fact))
```

Execute this with
```Shell
ipython --quiet multi_sin.py
```
instead of the bare `python` interpreter. This will make sure the script will work despite not being able to activate an actual plotting backend in script mode.

Let's try it and make a nice movie ...